In [1]:
# nuclio: ignore
import nuclio

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from mlrun.execution import MLClientCtx
from mlrun.datastore import DataItem
from mlrun.artifacts import PlotArtifact, TableArtifact
from mlrun.mlutils import gcf_clear

from yellowbrick import ClassBalance

from typing import List

/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
pd.set_option("display.float_format", lambda x: "%.2f" % x)

def summarize(
    context: MLClientCtx,
    table: str,
    label_column: str = "labels",
    class_labels: List[str] = [],
    plot_hist: bool = True,
    plots_dest: str = "plots"
) -> None:
    """Summarize a table

    :param context:         the function context
    :param table:           pandas dataframe (csv/parquet file path)
    :param label_column:    ground truth column label
    :param class_labels:    label for each class in tables and plots
    :param plot_hist:       (True) set this to False for large tables
    :param plots_dest:      destination folder of summary plots (relative to artifact_path)
    """
    table = str(table)
    if table.endswith(".csv"):
        table = pd.read_csv(table)
    else: 
        table = pd.read_parquet(table)
    header = table.columns.values
    
    gcf_clear(plt)
    snsplt = sns.pairplot(table, hue=label_column, diag_kws={"bw": 1.5})
    context.log_artifact(PlotArtifact("histograms",  body=plt.gcf()), local_path=f"{plots_dest}/hist.html")

    gcf_clear(plt)   
    labels = table.pop(label_column)
    if not class_labels:
        class_labels = labels.unique()
    class_balance_model = ClassBalance(labels=class_labels)
    class_balance_model.fit(labels)   
    scale_pos_weight = class_balance_model.support_[0]/class_balance_model.support_[1]
    context.log_result("scale_pos_weight", f"{scale_pos_weight:0.2f}")
    context.log_artifact(PlotArtifact("imbalance", body=plt.gcf()), local_path=f"{plots_dest}/imbalance.html")
    
    gcf_clear(plt)
    tblcorr = table.corr()
    ax = plt.axes()
    sns.heatmap(tblcorr, ax=ax, annot=False, cmap=plt.cm.Reds)
    ax.set_title("features correlation")
    context.log_artifact(PlotArtifact("correlation",  body=plt.gcf()), local_path=f"{plots_dest}/corr.html")
    # otherwise shows last plot:
    gcf_clear(plt)

In [4]:
# nuclio: end-code

In [5]:
from mlrun import run_local, code_to_function , NewTask, mlconf

mlconf.dbpath = "http://mlrun-api:8080"
mlconf.artifact_path = "/User/artifacts"

In [6]:
# create job function object from notebook code
fn = code_to_function("describe", kind="job", with_doc=True,
                      handler=summarize, image="mlrun/ml-models")

# add metadata (for templates and reuse)
fn.spec.default_handler = "summarize"
fn.spec.description = "describe and visualizes dataset stats"
fn.metadata.categories = ["models", "visualization"]
fn.metadata.labels = {"author": "yjb"}

fn.save()
fn.export("function.yaml")

[mlrun] 2020-03-31 14:29:34,047 saving function: describe, tag: latest
[mlrun] 2020-03-31 14:29:34,077 function spec saved to path: function.yaml


In [7]:
table_path = os.path.join(mlconf.artifact_path, "iris.parquet")
task = NewTask(name="tasks describe", handler=summarize, inputs={"table": table_path})
run = run_local(task)

[mlrun] 2020-03-31 14:29:34,092 starting run tasks describe uid=a28dddcd4c6e43089cd3412143bd8482  -> http://mlrun-api:8080


findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.


[mlrun] 2020-03-31 14:29:37,244 log artifact histograms at /User/artifacts/a28dddcd4c6e43089cd3412143bd8482/plots/hist.html, size: 177021, db: Y
[mlrun] 2020-03-31 14:29:37,825 log artifact imbalance at /User/artifacts/a28dddcd4c6e43089cd3412143bd8482/plots/imbalance.html, size: 7464, db: Y
[mlrun] 2020-03-31 14:29:38,011 log artifact correlation at /User/artifacts/a28dddcd4c6e43089cd3412143bd8482/plots/corr.html, size: 20942, db: Y



uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...bd8482,0,Mar 31 14:29:34,completed,tasks describe,v3io_user=adminkind=handlerowner=adminhost=jupyter-57954c4b4c-xln99,table,,scale_pos_weight=1.00,histogramsimbalancecorrelation


to track results use .show() or .logs() or in CLI: 
!mlrun get run a28dddcd4c6e43089cd3412143bd8482 --project default , !mlrun logs a28dddcd4c6e43089cd3412143bd8482 --project default
[mlrun] 2020-03-31 14:29:38,112 run executed, status=completed


In [8]:
import pandas as pd
df = pd.read_parquet(table_path)
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),labels
0,5.10,3.50,1.40,0.20,0.00
1,4.90,3.00,1.40,0.20,0.00
2,4.70,3.20,1.30,0.20,0.00
3,4.60,3.10,1.50,0.20,0.00
4,5.00,3.60,1.40,0.20,0.00
...,...,...,...,...,...
145,6.70,3.00,5.20,2.30,2.00
146,6.30,2.50,5.00,1.90,2.00
147,6.50,3.00,5.20,2.00,2.00
148,6.20,3.40,5.40,2.30,2.00
